In [1]:
from pydantic import BaseModel, Field
from typing import List
import instructor
from openai import OpenAI
from langchain_core.messages import AIMessage, ToolMessage

import json

from typing import List, Dict, Any, Annotated, Optional
from operator import add
from jinja2 import Template

In [2]:
# Pydantic Models

class ToolCall(BaseModel):
    name: str
    arguments: dict
    server: str

# from typing import Optional

# class ToolCall(BaseModel):
#     name: str
#     arguments: dict = Field(alias="parameters")
#     server: Optional[str] = None


class RAGUsedContext(BaseModel):
    id: int
    description: str

class AgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)
    final_answer: bool = Field(default=False)
    retrieved_context_ids: List[RAGUsedContext]

class QAAgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)
    final_answer: bool = Field(default=False)
    retrieved_context_ids: List[RAGUsedContext]

class ClassifierAgentResponse(BaseModel):
    answer: str
    tool_calls: List[ToolCall] = Field(default_factory=list)
    final_answer: bool = Field(default=False)


class Delegation(BaseModel):
    agent: str
    task: str = Field(default="")

class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: list[Delegation]
    final_answer: bool = Field(default=False)
    answer: str


class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    answer: str = ""
    iteration: int = Field(default=0)
    classifier_iteration: int = Field(default=0)
    coordinator_iteration: int = Field(default=0)
    job_posting_qa_final_answer: bool = Field(default=False)
    classifier_final_answer: bool = Field(default=False)
    coordinator_final_answer: bool = Field(default=False)
    qa_available_tools: List[Dict[str, Any]] = []
    classifier_available_tools: List[Dict[str, Any]] = []
    qa_tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    classifier_tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    retrieved_context_ids: List[RAGUsedContext] = []
    # NEW fields for classifier integration
    # retrieved_job_posting: Optional[str] = ""            # stores the retrieved job posting text
    classification_result: str = ""  # store fraud classification result
    user_intent: str = ""
    plan: list[Delegation] = Field(default_factory=list)
    next_agent: str = ""

In [3]:
def lc_messages_to_regular_messages(msg):

    if isinstance(msg, dict):
        
        if msg.get("role") == "user":
            return {"role": "user", "content": msg["content"]}
        elif msg.get("role") == "assistant":
            return {"role": "assistant", "content": msg["content"]}
        elif msg.get("role") == "tool":
            return {
                "role": "tool", 
                "content": msg["content"], 
                "tool_call_id": msg.get("tool_call_id")
            }
        
    elif isinstance(msg, AIMessage):

        result = {
            "role": "assistant",
            "content": msg.content
        }
        
        if hasattr(msg, 'tool_calls') and msg.tool_calls and len(msg.tool_calls) > 0 and not msg.tool_calls[0].get("name").startswith("functions."):
            result["tool_calls"] = [
                {
                    "id": tc["id"],
                    "type": "function",
                    "function": {
                        "name": tc["name"].replace("functions.", ""),
                        "arguments": json.dumps(tc["args"])
                    }
                }
                for tc in msg.tool_calls
            ]
            
        return result
    
    elif isinstance(msg, ToolMessage):

        return {"role": "tool", "content": msg.content, "tool_call_id": msg.tool_call_id}
    
    else:

        return {"role": "user", "content": str(msg)}

In [4]:
def coordinator_agent_node(state) -> dict:

    prompt_template = """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- job_posting_qa_agent: The user is asking a question about a job posting. This can be a question about its specifications, descriptions, text entities etc.
- classifier_agent: The user is asking to classifiy job postings on whether the posting is real or not.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true

Additional instructions:

- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

    prompt = Template(prompt_template).render()

    messages = state.messages

    conversation = []

    for msg in messages:
        conversation.append(lc_messages_to_regular_messages(msg))

    client = instructor.from_openai(OpenAI())

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1",
        response_model=CoordinatorAgentResponse,
        messages=[{"role": "system", "content": prompt}, *conversation],
        temperature=0,
    )

    if response.final_answer:
        ai_message = [AIMessage(
            content=response.answer,
        )]
    else:
        ai_message = []

    return {
        "messages": ai_message,
        "answer": response.answer,
        "next_agent": response.next_agent,
        "plan": response.plan,
        "coordinator_final_answer": response.final_answer,
        "coordinator_iteration": state.coordinator_iteration + 1,
        "trace_id": ""
    }

In [9]:
initial_state = State(messages=[{"role": "user", "content": "What is the weather today?"}])

In [10]:
answer = coordinator_agent_node(initial_state)

In [11]:
answer

{'messages': [AIMessage(content="I'm sorry, but I can only assist with questions related to job postings, such as their details, classifications, or authenticity. If you have a question about a job posting, please let me know!", additional_kwargs={}, response_metadata={})],
 'answer': "I'm sorry, but I can only assist with questions related to job postings, such as their details, classifications, or authenticity. If you have a question about a job posting, please let me know!",
 'next_agent': '',
 'plan': [],
 'coordinator_final_answer': True,
 'coordinator_iteration': 1,
 'trace_id': ''}

# Run Evaluation

In [5]:
from langsmith import Client
import os

ls_client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

In [12]:
def next_agent_evaluator(run, example):

    next_agent_match = run.outputs["next_agent"] == example.outputs["next_agent"]
    final_answer_match = run.outputs["coordinator_final_answer"] == example.outputs["coordinator_final_answer"]

    return next_agent_match and final_answer_match

In [15]:
results = ls_client.evaluate(
    lambda x: coordinator_agent_node(State(messages=x["messages"])),
    data="coordinator-evaluation-dataset",
    evaluators=[
        next_agent_evaluator
    ],
    experiment_prefix="coordinator-evaluation-dataset"
)

View the evaluation results for experiment: 'coordinator-evaluation-dataset-0aa25b56' at:
https://smith.langchain.com/o/cd0c2f03-d905-4ac2-af13-691ee0bcd17d/datasets/956ca4fa-0eaf-4c3c-897e-2eeda3d3fac9/compare?selectedSessions=f87a0803-cbd6-4071-bfff-4d8f886f4a74




10it [00:10,  1.03s/it]


In [14]:
results

,inputs.messages,outputs.messages,outputs.answer,outputs.next_agent,outputs.plan,outputs.coordinator_final_answer,outputs.coordinator_iteration,outputs.trace_id,error,reference.next_agent,reference.coordinator_final_answer,feedback.next_agent_evaluator,execution_time,example_id,id
0,"[{'role': 'user', 'content': 'What are the ent...",[],"To answer your query, I will extract all the e...",job_posting_qa_agent,[agent='job_posting_qa_agent' task='Extract al...,False,1,,None,job_posting_qa_agent,False,True,1.295042,a3422d36-1309-4be9-83cf-4212a9433b6d,ee7f3299-8bea-48a4-9d44-b71e3f86c1f5
1,"[{'role': 'user', 'content': 'Is job 123 relat...",[],I will check the description and requirements ...,job_posting_qa_agent,[agent='job_posting_qa_agent' task='Check if j...,False,1,,None,job_posting_qa_agent,False,True,1.635403,615fa3e2-d6fe-4e02-94da-bd1b13720379,b05cb4b3-c72b-4fea-8120-981b5f2b215d
2,"[{'role': 'user', 'content': 'Can you find me ...",[],"To find 2 suspicious data job postings, I will...",classifier_agent,[agent='classifier_agent' task='Classify data ...,False,1,,None,job_posting_qa_agent,False,False,1.434492,927cfadf-4a09-4e20-8003-bfcc0fbf3952,84b34441-7817-43a7-95b3-273f2484132f
3,"[{'role': 'user', 'content': 'Can you help me ...",[content='Of course! Please provide more detai...,Of course! Please provide more details about y...,,[],True,1,,None,,True,True,1.020481,64d398c6-40ee-470d-bf55-7cd4aa7dfa08,4bff5b22-2eff-4e0b-b1e8-51d4aa451fa2
4,"[{'role': 'user', 'content': 'What of kind thi...","[content=""I can classify job postings based on...",I can classify job postings based on their aut...,,[agent='classifier_agent' task='List the types...,True,1,,None,,True,True,1.118441,00edd68e-0f8c-4a65-b37d-3445fb8af921,08670659-59d2-45ae-a11b-2a07eee9a5b9
5,"[{'role': 'user', 'content': 'Is job 123 a fra...",[],"First, classify whether job 123 is a fraudulen...",classifier_agent,[agent='classifier_agent' task='Classify wheth...,False,1,,None,classifier_agent,False,True,1.372148,b7b75d11-49f4-4c50-b2a0-b6dd6614977f,204aeddf-b021-4809-9ddd-0393d04ea06f
6,"[{'role': 'user', 'content': 'Can you classify...",[],"To classify multiple job postings, I will dele...",classifier_agent,[agent='classifier_agent' task='Classify the p...,False,1,,None,,True,False,0.880361,5b426dbb-fee2-4878-a57b-6883f13756f9,f1ddb674-6c86-4663-88a8-15f3caf6ea3a
7,"[{'role': 'user', 'content': 'Can you classify...",[],I will classify the job posting with ID 10397 ...,classifier_agent,[agent='classifier_agent' task='Classify the j...,False,1,,None,classifier_agent,False,True,1.020870,1b2460ea-2f24-49be-af0f-8cd6e42d648c,ae018a48-8900-427e-ab3c-88e59fc0008b
8,"[{'role': 'user', 'content': 'What is job 1039...",[],I will retrieve and summarize the details and ...,job_posting_qa_agent,[agent='job_posting_qa_agent' task='Retrieve a...,False,1,,None,job_posting_qa_agent,False,True,1.120969,317baa90-3826-4f6b-87c7-c2582523fd2c,0e5d7e24-c594-4315-b11e-d764df8fbbd1
9,"[{'role': 'user', 'content': 'Whats is the wea...","[content=""I'm here to assist with job postings...",I'm here to assist with job postings and relat...,,[],True,1,,None,,True,True,1.759363,d96d7c6a-8be4-4393-80c1-3d3f34b5c4d7,f2776a18-f04c-4b13-8d1d-ca98e0694d35


In [16]:
results_resp = ls_client.read_project(
    project_name = results.experiment_name,
    include_stats=True
)

In [17]:
results_resp.feedback_stats

{'next_agent_evaluator': {'n': 10,
  'avg': 0.9,
  'stdev': 0.30000000000000004,
  'errors': 0,
  'values': {}}}

In [18]:
results_resp.feedback_stats['next_agent_evaluator']['avg']

0.9